In [1]:
import numpy as np
import pandas as pd

import ipywidgets as widgets
from IPython.display import display, Math

In [2]:
import bokeh.plotting as bplt
from bokeh.models import Range1d, LabelSet, ColumnDataSource
from bokeh.models.glyphs import Step, Line
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

# The Random Walk: Section 3 - Reflecting Barriers

In the simple random walk (every step was up or down by 1), we had an infinite state space Markov chain that was either transient (for $p \neq 0.5$) or null recurrent ($p = 0.5$).  There was no value of $p$ that allowed for a positive recurrent random walk.

To see an example of an infinite state space Markov chain that is positive recurrent, we will add a “reflecting barrier” to the random walk.  In particular, we restrict the state space to the *non-negative* integers $\mathcal{S} = \{0, 1, 2, 3, \dots\}$ and make $p_{00} = 1 - p$.

For any state $i \geq 1$, the behavior is exactly the same as the simple random walk:
$$p_{ij} = \begin{cases}
    p & j = i + 1 \\
    1 - p & j = i - 1 \\
    0 & \text{otherwise (i.e., any other state $j$)}
  \end{cases}$$
only the behavior for state $0$ changes:
$$p_{01} = p \qquad p_{00} = 1 - p$$
If you were to write (part of) this in the usual matrix form, it would be:
$$\mathbf{P} = \begin{pmatrix}
    1-p & p & 0 & 0 & \dots \\
    1-p & 0 & p & 0 & \dots \\
    0 & 1-p & 0 & p & \dots \\
    0 & 0 & 1-p & 0 & \dots \\
    \vdots & \vdots & \vdots & \vdots & \ddots \\
  \end{pmatrix}$$


## Positive Recurrence for $p < 0.5$

As with the original random walk, you should (quickly) check that this Markov chain is irreducible.

We will show that, for $p < 0.5$, the Markov chain is positive recurrent.  To do so, we will rely on the theorem from Section 6 of Discrete-Time Markov Chains, which we restate here.

**Theorem:** An *irreducible* Markov chain is positive recurrent if and only if there exists a unique solution to:
$$ \pi = \pi \mathbf{P} $$
with $\pi \geq 0$ and $\sum_j \pi_j = 1$.

So if we can find a solution $\pi$ that satisfies the above, then the Markov chain is positive recurrent.  The basic steps will be the same as when we had a finite state space:
* Write a system of equations.
* Solve for $\pi$.

However, we will need a way to handle the fact that there are an infinite number of equations and elements of $\pi$ to solve.  We will do so by identifying a recursion relationship and applying the [principle of mathematical induction](https://en.wikipedia.org/wiki/Mathematical_induction).

We will start by listing the system of equations $\pi = \pi \mathbf{P}$:
$$\begin{aligned}
    \pi_0 &= (1 - p) \pi_0 + (1 - p) \pi_1 \\
    \pi_1 &= p \pi_0 + (1 - p) \pi_2 \\
    \pi_2 &= p \pi_1 + (1 - p) \pi_3 \\
    &\,\,\vdots \\
    \pi_i &= p \pi_{i-1} + (1 - p) \pi_{i+1} \qquad i \geq 1
  \end{aligned}$$
  
**Base Case:** The first equation (for $\pi_0$) yields a relationship between state $1$ and state $0$:
$$\begin{aligned}
    \pi_0 &= (1 - p) \pi_0 + (1 - p) \pi_1 \\
    p \pi_0 &= (1 - p) \pi_1 \\
  \end{aligned}$$
Combining this with the second equation (for $\pi_1$) yields:
$$\begin{aligned}
    \pi_1 &= p \pi_0 + (1 - p) \pi_2 \\
    \pi_1 &= (1 - p) \pi_1 + (1 - p) \pi_2 \\
    p \pi_1 &= (1 - p) \pi_2
  \end{aligned}$$
**Inductive Step:** We see that this pattern repeats in the sense that, if the pattern holds for states $i-1$ and $i$ ($p \pi_{i-1} = (1 - p) \pi_i$) then
$$\begin{aligned}
    \pi_i &= p \pi_{i-1} + (1 - p) \pi_{i+1} \\
    \pi_i &= (1 - p) \pi_i + (1 - p) \pi_{i+1} \\
    p \pi_i &= (1 - p) \pi_{i+1}
  \end{aligned}$$
the pattern holds for states $i$ and $i + 1$.

Therefore, we have the following *recursion*, for all $i \geq 0$:
$$\pi_{i+1} = \frac{p}{1-p} \pi_i$$

Building this up from $i = 0$:
$$\begin{aligned}
    \pi_1 &= \frac{p}{1 - p} \pi_0 \\
    \pi_2 &= \frac{p}{1 - p} \pi_1 = \left(\frac{p}{1 - p}\right)^2 \pi_0 \\
    \pi_3 &= \frac{p}{1 - p} \pi_2 = \left(\frac{p}{1 - p}\right)^3 \pi_0 \\
    &\,\,\vdots \\
    \pi_i &= \frac{p}{1 - p} \pi_{i-1} = \left(\frac{p}{1 - p}\right)^i \pi_0 \\
  \end{aligned}$$

At this point we only have one “unknown” to solve for: $\pi_0$.  Fortunately, we have another equation to use, $\sum_j \pi_j = 1$.
$$\begin{aligned}
    \sum_{i=0}^\infty \pi_i &= 1 \\
    \sum_{i=0}^\infty \left(\frac{p}{1 - p}\right)^i \pi_0 &= 1 \\
    \pi_0 \left(\sum_{i=0}^\infty \left(\frac{p}{1 - p}\right)^i\right) &= 1 \\
    \pi_0 \left(\frac{1}{1 - \frac{p}{1-p}}\right) &= 1 \\
    \pi_0 &= 1 - \frac{p}{1-p} \\
  \end{aligned}$$

In summary, the stationary distribution for the reflected random walk ($p < 0.5$) is:
$$ \pi_i = \left(\frac{p}{1 - p}\right)^i \left(1 - \frac{p}{1-p}\right) \qquad \text{for any } i \in \mathcal{S} = \{0, 1, 2, 3, \dots\}$$


In [3]:
p = 0.7
steps = 1000
X_0 = 0
delta = np.random.choice([-1,1], size=[1,steps], p=[1-p, p])
sample_path = pd.DataFrame(np.append(np.array(X_0), np.cumsum(delta)), index=pd.Index(range(0,steps+1)), columns=['Random Walk'])
sample_path['Regulator'] = sample_path['Random Walk'].cummin()
sample_path['Reflected Random Walk'] = sample_path['Random Walk'] - sample_path['Regulator']

In [4]:
plt = bplt.figure(title='Reflected Random Walk')
plt.title.align = 'center'
plt.title.text_font_size = '25px'
source = ColumnDataSource(sample_path)
glyph = Line(x="index", y="Reflected Random Walk", line_color="blue")
plt.add_glyph(source, glyph)
bplt.show(plt)

## The Regulator Process (Optional)

To simulate the reflected random walk, we did not (directly) simulate the Markov chain as presented in the beginning of the section.  Rather, we simulated a (unreflected) random walk with the same $p$ and then constructed the reflected random walk via the “regulator process.”

In particular:
* Let $\{X_n,\, n \geq 0\}$ be the (unreflected) random walk.
* Let $\{Y_n,\, n \geq 0\}$ be:
$$ Y_n = -\min\{0, X_0,\dots,X_n\} $$
* Then $X_n + Y_n$ is the reflected random walk.

## Deriving the Summation Formula (Optional)

In both the Gambler's Ruin and the Reflected Random Walk, we made use of the following summation formula (for non-negative $x$):

$$\sum_{i=0}^n x^i = \frac{1 - x^{n+1}}{1 - x} \qquad x \neq 1$$

This is not very difficult to derive (and may be easier than memorizing it).  It suffices to show: 

$$(1 - x) \sum_{i=0}^n x^i = 1 - x^{n+1}$$

because we can then divide both sides by $(1 - x)$ to get the desired summation formula (this is the step that requires $x \neq 1$).

The key is to recognize a [*telescoping series*](https://en.wikipedia.org/wiki/Telescoping_series):

$$\require{cancel}
  \begin{aligned}
    (1 - x) \sum_{i=0}^n x^i &= \sum_{i=0}^n (x^i - x^{i+1}) \\
    &= (1 - x) + (x - x^2) + \dots + (x^{n-1} - x^n) + (x^n - x^{n+1}) \\
    &= 1 - x^{n+1}
  \end{aligned}$$
  
In the case of the reflected random walk, we were faced with the infinite sum, and needed to take the limit as $n \to \infty$.  The summation converges (is finite) only for $x < 1$:

$$\sum_{i=0}^\infty x^i = \lim_{n \to \infty} \sum_{i=0}^n x^i = \lim_{n \to \infty} \frac{1 - x^{n+1}}{1 - x} = \frac{1 - \lim_{n \to \infty} x^{n+1}}{1 - x} = \frac{1}{1 - x}$$